In [239]:
# importing packages
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.lines as mlines
from matplotlib.gridspec import GridSpec

%matplotlib qt

from scipy.optimize import curve_fit
from scipy.special import expit

import ssm
from ssm import transitions 

import scipy.io
import os
import seaborn as sb
import pandas as pd

In [240]:
# load data
filename = 'C:/Users/Asus/Desktop/PhD/rotations/AnnJeff/data/riskdata/animalspaper.mat'
data = scipy.io.loadmat(filename)
data = data['riskydata']
df = pd.DataFrame(data)
df.columns = ['animal', 'sessionid', 'trialnumber', 'trial_block', 'lotterymag', 'lotteryprob', 
           'surebetmag', 'rewardreceived', 'lotterychoice', 'lotteryoutcome']

In [306]:
# defining plot colors
stimcolors = sb.color_palette("rocket_r")
colors = ['gold', 'plum', 'lightskyblue']

In [241]:
# getting relevant variables from data
animals = df.animal.unique()
nanimals = len(animals)

In [244]:
# normalization function
def normV(X, method):   
    if method == 'max':
        return X/max(X)
    if method == 'zscore':
        return (X - np.mean(X))/np.std(X)

In [292]:
# creating input structure compatible with the ssm package
inputdim = 2
inputs = []
Y = []

for aa in range(nanimals):
    animaldata = df[df.animal == animals[aa]]
    sessions = animaldata.sessionid.unique()
    nsessions = len(sessions)
    
    inputsaux = []
    yaux = []
    for ss in range(nsessions):
        sessiondata = animaldata[animaldata.sessionid == sessions[ss]]
        ntrials = len(sessiondata)
        inputsaux.append(np.ones([ntrials, inputdim]))
        yaux.append(np.zeros([ntrials,1], dtype = int))
        
        sbmag = sessiondata.surebetmag[sessiondata.lotterychoice == 0].unique()
        sbrwd = sessiondata.rewardreceived[sessiondata.lotterychoice == 0].unique()
        rwdmult = sbrwd/sbmag
        lotteryprob = sessiondata.lotteryprob.unique()
        
        deltaEV = rwdmult * (sessiondata.lotterymag * lotteryprob) - sbrwd
        normdeltaEV = normV(deltaEV, 'max')
        inputsaux[ss][:,0] = normdeltaEV
        yaux[ss][:,0] = sessiondata.lotterychoice
    
    inputs.append(inputsaux)
    Y.append(yaux)

In [369]:
# setting glm-hmm parameters
nstates = 3
obsdim = 1
inputdim = 2
ncatergories = 2
niterations = 200
# creating new df variables for estimated states from glmhmm
df['pstate1'] = np.zeros(len(df))
df['pstate2'] = np.zeros(len(df))
df['pstate3'] = np.zeros(len(df))
df['state'] = np.zeros(len(df))

# creating variables to keep from HMM
w = []
transmat = []

for aa in range(nanimals):
    w.append(np.zeros([nstates, inputdim]))
    transmat.append(np.zeros([nstates, nstates]))
    
    glmhmm = ssm.HMM(nstates, obsdim, inputdim,  observations = 'input_driven_obs',
                observation_kwargs = dict(C = ncatergories), transitions = "standard")
    glmhmm2fit = glmhmm.fit(Y[aa], inputs = inputs[aa], method = 'em', num_iters = niterations, tolerance = 10**-5)
    
    # saving the glm weights and transition matrix values
    waux = - glmhmm.observations.params.reshape(nstates,inputdim)
    sortidcs = waux[:, 1].argsort()
    w[aa] = waux[sortidcs]
    
    transmat[aa] = glmhmm.transitions.transition_matrix[sortidcs]
    
    # getting expected states
    posterior_pstate = [glmhmm.expected_states(data = data, input = inputs)[0]
                        for data, inputs
                        in zip(Y[aa], inputs[aa])]
   
    # saving expected states  for each trialin the data frame
    ppstate_aux = np.array([item for sublist in posterior_pstate for item in sublist])
    ppstate_aux[:, sortidcs]
    df.loc[df.animal == animals[aa],['pstate1', 'pstate2','pstate3']] = ppstate_aux

df.state = df[['pstate1', 'pstate2', 'pstate3']].idxmax(axis = 1)    

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [370]:
# plot psychometric curves & glm weights
for aa in range(nanimals):
    fig, ax = plt.subplots(1,2)
    fig.suptitle('animal: ' + str(animals[aa]))
    
    evs = np.unique(inputs[aa][0][:,0])
    evspace = np.linspace(np.min(evs), np.max(evs), 1000)
    X = np.array([
        evspace,
        np.repeat(1, len(evspace))]).T
    
    for kk in range(nstates):       
        # plot psycurves
        plt.subplot(1,2,1)
        
        glmweights = w[aa][kk, :]
        wX = np.matmul(X, glmweights)
        
        plt.plot(evspace, expit(wX), lw = 2.5, color = colors[kk])
        
        # reference lines
        plt.axvline(0, linestyle = ':', c = 'k',  lw = .5)
        plt.axhline(.5, linestyle = ':', c = 'k', lw = .5)
    
        # axis options
        plt.ylim((-.05, 1.05))
        plt.xlabel('$\Delta$ EV')  
        plt.ylabel('p(risk)')
        ax[0].set_yticks([0, .5, 1])
        plt.xticks(fontsize = 7.5)
        plt.yticks(fontsize = 7.5)
        ax[0].spines['top'].set_visible(False)
        ax[0].spines['right'].set_visible(False)
        
        # plot glm weights for each covariate
        plt.subplot(1,2,2)
        plt.plot(range(inputdim), glmweights, marker = 'o', 
                 color = colors[kk], alpha = .7, linestyle = '-', label = 'state ' + str(kk+1))

        plt.ylabel('glm weight')
        plt.xticks(range(inputdim), ['$\Delta$ EV ', 'bias'])
        plt.axhline(y = 0, color = 'k', alpha = .5, ls = '--')
        plt.legend()
        ax[1].spines['top'].set_visible(False)
        ax[1].spines['right'].set_visible(False)